In [1]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output, State
import pandas as pd
import numpy as np
import plotly.express as px
import sqlite3
from sqlalchemy import create_engine
import plotly.graph_objects as go

#### SQL Queries

In [3]:
engine = create_engine('sqlite:///eriecountydb')

In [4]:
sql_statement = """
    SELECT 
        Zipcode
    FROM Location
    ORDER BY Zipcode ASC
    """
zipcode_query = pd.read_sql_query(sql_statement, engine)

In [5]:
def calculate_stats(zipcode):
    sql_statement = f"""
    SELECT 
        Food_Store.Zipcode,
        COUNT(Food_Store.Store_Name) AS Number_of_Stores,
        Location.City
    FROM Food_Store
    JOIN Location ON Food_Store.Zipcode = Location.Zipcode
    WHERE Food_Store.Zipcode = {zipcode}
    ORDER BY Number_of_Stores ASC
    """
    stats_query = pd.read_sql_query(sql_statement, engine)
    
    sql_statement = f"""
    SELECT 
        Income_Total.No_of_Returns AS Approx_Population
    FROM Income_Total
    WHERE Zipcode = {zipcode}
    """
    stats_query2 = pd.read_sql_query(sql_statement, engine)
    zip_name = stats_query['City'].iloc[0]
    number_of_stores = stats_query['Number_of_Stores'].iloc[0] if not stats_query.empty else 0
    approximate_population = stats_query2['Approx_Population'].iloc[0] if not stats_query2.empty else 'N/A'
    return {'Name' : zip_name.capitalize(),
            'Number_of_Stores' : number_of_stores,
           'Approximate Population' : approximate_population}


In [6]:
sql_statement = '''SELECT 
    loc.Zipcode,
    Income_Under_25k.No_of_Returns AS Returns_Under_25k,
    Income_25k_to_50k.No_of_Returns AS Returns_25k_to_50k,
    Income_50k_to_75k.No_of_Returns AS Returns_50k_to_75k,
    Income_75k_to_100k.No_of_Returns AS Returns_75k_to_100k,
    Income_100k_to_200k.No_of_Returns AS Returns_100k_to_200k,
    Income_Above_200k.No_of_Returns AS Returns_Above_200k
FROM 
    (SELECT DISTINCT Zipcode FROM Income_Under_25k
     UNION
     SELECT DISTINCT Zipcode FROM Income_25k_to_50k
     UNION
     SELECT DISTINCT Zipcode FROM Income_50k_to_75k
     UNION
     SELECT DISTINCT Zipcode FROM Income_75k_to_100k
     UNION
     SELECT DISTINCT Zipcode FROM Income_100k_to_200k
     UNION
     SELECT DISTINCT Zipcode FROM Income_Above_200k) AS Zipcodes
LEFT JOIN Location AS loc ON Zipcodes.Zipcode = loc.Zipcode
LEFT JOIN Income_Under_25k ON Income_Under_25k.Zipcode = loc.Zipcode
LEFT JOIN Income_25k_to_50k ON Income_25k_to_50k.Zipcode = loc.Zipcode
LEFT JOIN Income_50k_to_75k ON Income_50k_to_75k.Zipcode = loc.Zipcode
LEFT JOIN Income_75k_to_100k ON Income_75k_to_100k.Zipcode = loc.Zipcode
LEFT JOIN Income_100k_to_200k ON Income_100k_to_200k.Zipcode = loc.Zipcode
LEFT JOIN Income_Above_200k ON Income_Above_200k.Zipcode = loc.Zipcode;
    '''
erie_income_query = pd.read_sql_query(sql_statement, engine)

In [7]:
sql_statement = '''SELECT 
    loc.Zipcode,
    Income_Under_25k.No_of_Returns AS Returns_Under_25k,
    Income_25k_to_50k.No_of_Returns AS Returns_25k_to_50k,
    Income_50k_to_75k.No_of_Returns AS Returns_50k_to_75k,
    Income_75k_to_100k.No_of_Returns AS Returns_75k_to_100k,
    Income_100k_to_200k.No_of_Returns AS Returns_100k_to_200k,
    Income_Above_200k.No_of_Returns AS Returns_Above_200k
FROM 
    (SELECT DISTINCT Zipcode FROM Income_Under_25k
     UNION
     SELECT DISTINCT Zipcode FROM Income_25k_to_50k
     UNION
     SELECT DISTINCT Zipcode FROM Income_50k_to_75k
     UNION
     SELECT DISTINCT Zipcode FROM Income_75k_to_100k
     UNION
     SELECT DISTINCT Zipcode FROM Income_100k_to_200k
     UNION
     SELECT DISTINCT Zipcode FROM Income_Above_200k) AS Zipcodes
LEFT JOIN Location AS loc ON Zipcodes.Zipcode = loc.Zipcode
LEFT JOIN Income_Under_25k ON Income_Under_25k.Zipcode = loc.Zipcode
LEFT JOIN Income_25k_to_50k ON Income_25k_to_50k.Zipcode = loc.Zipcode
LEFT JOIN Income_50k_to_75k ON Income_50k_to_75k.Zipcode = loc.Zipcode
LEFT JOIN Income_75k_to_100k ON Income_75k_to_100k.Zipcode = loc.Zipcode
LEFT JOIN Income_100k_to_200k ON Income_100k_to_200k.Zipcode = loc.Zipcode
LEFT JOIN Income_Above_200k ON Income_Above_200k.Zipcode = loc.Zipcode
WHERE loc.City = "BUFFALO";
    '''
city_income_query = pd.read_sql_query(sql_statement, engine)

In [8]:
sql_statement = '''SELECT 
    loc.Zipcode,
    Income_Under_25k.No_of_Returns AS Returns_Under_25k,
    Income_25k_to_50k.No_of_Returns AS Returns_25k_to_50k,
    Income_50k_to_75k.No_of_Returns AS Returns_50k_to_75k,
    Income_75k_to_100k.No_of_Returns AS Returns_75k_to_100k,
    Income_100k_to_200k.No_of_Returns AS Returns_100k_to_200k,
    Income_Above_200k.No_of_Returns AS Returns_Above_200k
FROM 
    (SELECT DISTINCT Zipcode FROM Income_Under_25k
     UNION
     SELECT DISTINCT Zipcode FROM Income_25k_to_50k
     UNION
     SELECT DISTINCT Zipcode FROM Income_50k_to_75k
     UNION
     SELECT DISTINCT Zipcode FROM Income_75k_to_100k
     UNION
     SELECT DISTINCT Zipcode FROM Income_100k_to_200k
     UNION
     SELECT DISTINCT Zipcode FROM Income_Above_200k) AS Zipcodes
LEFT JOIN Location AS loc ON Zipcodes.Zipcode = loc.Zipcode
LEFT JOIN Income_Under_25k ON Income_Under_25k.Zipcode = loc.Zipcode
LEFT JOIN Income_25k_to_50k ON Income_25k_to_50k.Zipcode = loc.Zipcode
LEFT JOIN Income_50k_to_75k ON Income_50k_to_75k.Zipcode = loc.Zipcode
LEFT JOIN Income_75k_to_100k ON Income_75k_to_100k.Zipcode = loc.Zipcode
LEFT JOIN Income_100k_to_200k ON Income_100k_to_200k.Zipcode = loc.Zipcode
LEFT JOIN Income_Above_200k ON Income_Above_200k.Zipcode = loc.Zipcode
WHERE loc.City != "BUFFALO";
    '''
sub_income_query = pd.read_sql_query(sql_statement, engine)

In [9]:
sql_statement = """
            SELECT 
                AVG(Store_Count) AS Average_Stores_Per_Zipcode
            FROM (
                SELECT 
                    Zipcode, 
                    COUNT(Store_ID) AS Store_Count
                FROM 
                    Food_Store
                GROUP BY 
                    Zipcode
            ) AS ZipcodeStoreCounts;
            """
erie_avg = pd.read_sql_query(sql_statement, engine)

In [10]:
sql_statement = """
            SELECT 
            AVG(Store_Count) AS Average_Stores_Per_Zipcode
        FROM (
            SELECT 
                Food_Store.Zipcode AS Store_Zipcode, 
                Location.City,
                COUNT(Store_ID) AS Store_Count
            FROM 
                Food_Store
            JOIN Location ON Food_Store.Zipcode = Location.Zipcode
            WHERE Location.City = "BUFFALO"
            GROUP BY 
                Food_Store.Zipcode
        ) AS ZipcodeStoreCounts;
            """
city_avg = pd.read_sql_query(sql_statement, engine)

In [11]:
sql_statement = """
                            SELECT 
                            AVG(Store_Count) AS Average_Stores_Per_Zipcode
                        FROM (
                            SELECT 
                                Food_Store.Zipcode AS Store_Zipcode, 
                                Location.City,
                                COUNT(Store_ID) AS Store_Count
                            FROM 
                                Food_Store
                            JOIN Location ON Food_Store.Zipcode = Location.Zipcode
                            WHERE Location.City != "BUFFALO"
                            GROUP BY 
                                Food_Store.Zipcode
                        ) AS ZipcodeStoreCounts;
                        """
sub_avg = pd.read_sql_query(sql_statement, engine)

In [12]:
sql_statement = """
        SELECT 
            Food_Store.Store_Focus,
            COUNT(Food_Store.Store_Name) AS Number_of_Stores
        FROM Food_Store
        JOIN Location ON Food_Store.Zipcode = Location.Zipcode
        WHERE Location.City = "BUFFALO"
        GROUP BY Store_Focus
        ORDER BY Number_of_Stores ASC
    """
focus_city = pd.read_sql_query(sql_statement, engine)

In [13]:
sql_statement = """
        SELECT 
            Food_Store.Store_Focus,
            COUNT(Food_Store.Store_Name) AS Number_of_Stores
        FROM Food_Store
        JOIN Location ON Food_Store.Zipcode = Location.Zipcode
        WHERE Location.City != "BUFFALO"
        GROUP BY Store_Focus
        ORDER BY Number_of_Stores ASC
    """
focus_sub = pd.read_sql_query(sql_statement, engine)

#### Dashboard

In [29]:
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Food Accessibility in Erie County, NY"),
    html.Div([
        html.Label("Select Zipcode:"),
        dcc.Dropdown(
            id='zip-dropdown',
            options=[{'label': zipc, 'value': zipc} for zipc in zipcode_query['Zipcode']],
            value='14001'
        )
    ], style={'width': '45%', 'display': 'inline-block'}),
    html.Div([
        html.H4("Statistics for Selected Zipcode:"),
        html.Div(id='zip-stats'),
    ], style={'width': '50%', 'display': 'inline-block', 'verticalAlign': 'top'}),
    html.Div([
        dcc.Graph(id='piechart')
    ], style={'width': '100%', 'display': 'inline-block'}),
    html.Div([
        dcc.Graph(id='barchart')
    ], style={'width': '100%', 'display': 'inline-block'}),
    html.Hr(),
    html.H2("Erie County Regional Statistics"),
    html.Div(id='model-equation', style={'fontSize': 18, 'fontWeight': 'bold'}),
    html.Div([
        html.Label("Select to view statistics: "),
        html.Button('Erie County', id='county-button', n_clicks=0),
        html.Div(id='county-output', style={'marginTop': '20px', 'fontSize': 16})
    ], style={'width': '50%', 'display': 'inline-block'}),
     html.Div([
        dcc.Graph(id='barchart-county')
    ], style={'width': '100%', 'display': 'inline-block'}),
    html.Div([
        html.Label("Select to view statistics: "),
        html.Button('City of Buffalo', id='city-button', n_clicks=0),
        html.Div(id='city-output', style={'marginTop': '20px', 'fontSize': 16})
    ], style={'width': '50%', 'display': 'inline-block'}),
    html.Div([
        dcc.Graph(id='barchart-city')
    ], style={'width': '100%', 'display': 'inline-block'}),
    html.Div([
        html.Label("Select to view statistics: "),
        html.Button('Suburbs', id='suburb-button', n_clicks=0),
        html.Div(id='suburb-output', style={'marginTop': '20px', 'fontSize': 16})
    ], style={'width': '50%', 'display': 'inline-block'}),
     html.Div([
        dcc.Graph(id='barchart-suburb')
    ], style={'width': '100%', 'display': 'inline-block'}),
    html.Div([
        html.H1("Quality of Food Stores by Region"),
        dcc.RadioItems(
                ['City of Buffalo', 'Suburbs'],
                'City of Buffalo',
                id='last-buttons',
                labelStyle={'display': 'inline-block', 'marginTop': '5px'})
    ],style={'width': '49%', 'display': 'inline-block'}),
    html.Div([
        dcc.Graph(id='last-graph')
    ], style={'width': '100%', 'display': 'inline-block'}),
])

@app.callback(
    [Output('zip-stats', 'children'),
     Output('piechart', 'figure'),
    Output('barchart', 'figure')],
    Input('zip-dropdown', 'value')
)
def update_piechart(zipcode):
    sql_statement = f"""
        SELECT 
            Food_Store.Store_Focus,
            COUNT(Food_Store.Store_Name) AS Number_of_Stores
        FROM Food_Store
        WHERE Zipcode = {zipcode}
        GROUP BY Store_Focus
    """
    piechart_query = pd.read_sql_query(sql_statement, engine)
    fig = px.pie(piechart_query, values='Number_of_Stores', names='Store_Focus', title=f"Store Focus Distribution for Zipcode {zipcode}")
    sql_statement = f""" SELECT
        loc.Zipcode,
        Income_Under_25k.No_of_Returns AS Under_25k,
        Income_25k_to_50k.No_of_Returns AS _25k_to_50k,
        Income_50k_to_75k.No_of_Returns AS _50k_to_75k,
        Income_75k_to_100k.No_of_Returns AS _75k_to_100k,
        Income_100k_to_200k.No_of_Returns AS _100k_to_200k,
        Income_Above_200k.No_of_Returns AS Above_200k
    FROM 
        (SELECT DISTINCT Zipcode FROM Income_Under_25k
         UNION
         SELECT DISTINCT Zipcode FROM Income_25k_to_50k
         UNION
         SELECT DISTINCT Zipcode FROM Income_50k_to_75k
         UNION
         SELECT DISTINCT Zipcode FROM Income_75k_to_100k
         UNION
         SELECT DISTINCT Zipcode FROM Income_100k_to_200k
         UNION
         SELECT DISTINCT Zipcode FROM Income_Above_200k) AS Zipcodes
    LEFT JOIN Location AS loc ON Zipcodes.Zipcode = loc.Zipcode
    LEFT JOIN Income_Under_25k ON Income_Under_25k.Zipcode = loc.Zipcode
    LEFT JOIN Income_25k_to_50k ON Income_25k_to_50k.Zipcode = loc.Zipcode
    LEFT JOIN Income_50k_to_75k ON Income_50k_to_75k.Zipcode = loc.Zipcode
    LEFT JOIN Income_75k_to_100k ON Income_75k_to_100k.Zipcode = loc.Zipcode
    LEFT JOIN Income_100k_to_200k ON Income_100k_to_200k.Zipcode = loc.Zipcode
    LEFT JOIN Income_Above_200k ON Income_Above_200k.Zipcode = loc.Zipcode
        WHERE loc.Zipcode = {zipcode}
    """
    barchart_query = pd.read_sql_query(sql_statement, engine)
    barchart_query_long = pd.melt(barchart_query, id_vars=['Zipcode'],
                              value_vars=['Under_25k', '_25k_to_50k', '_50k_to_75k', '_75k_to_100k', '_100k_to_200k', 'Above_200k'],
                              var_name='Income_Bracket', value_name='No_of_Returns')
    fig2 = px.bar(barchart_query_long, 
              x='Income_Bracket',  # The income range as the x-axis
              y='No_of_Returns',  # The number of returns as the y-axis
              title=f"Income Distribution for Zipcode {zipcode}", 
              labels={'Income_Bracket': 'Income Range', 'No_of_Returns': 'Number of Returns'})
    stats_text = calculate_stats(zipcode)
    stats_result = [f"{key}: {value}" for key, value in stats_text.items()]
    return html.Ul([html.Li(stat) for stat in stats_result]), \
            fig, fig2

@app.callback(
    [Output('county-output', 'children'),
    Output('barchart-county', 'figure')],
    Input('county-button', 'n_clicks'),
)
def update_county_stats(n_clicks):
    erie_dict = {"Average_Stores_Per Zipcode": "No Data Available"}
    fig = go.Figure() 
    if n_clicks > 0:
        erie_dict = {'Average_Stores_Per Zipcode': erie_avg['Average_Stores_Per_Zipcode'].iloc[0]}
        fig = px.bar(erie_income_query, 
                        x=['Returns_Under_25k', 'Returns_25k_to_50k', 'Returns_50k_to_75k', 'Returns_75k_to_100k', 'Returns_100k_to_200k', 'Returns_Above_200k'],
                        y='Zipcode', title="Income Distribution per Zipcode in Erie County")
    return html.Ul([html.Li(f"{key}: {value}") for key, value in erie_dict.items()]), fig


@app.callback(
    [Output('city-output', 'children'),
    Output('barchart-city', 'figure')],
    Input('city-button', 'n_clicks'),
)
def update_city_stats(n_clicks):
    city_dict = {"Average_Stores_Per Zipcode": "No Data Available"}
    fig = go.Figure() 
    if n_clicks > 0:
        city_dict = {'Average_Stores_Per Zipcode': city_avg['Average_Stores_Per_Zipcode'].iloc[0]}
        fig = px.bar(city_income_query, x=['Returns_Under_25k', 'Returns_25k_to_50k', 'Returns_50k_to_75k', 'Returns_75k_to_100k', 'Returns_100k_to_200k', 'Returns_Above_200k'],
                        y='Zipcode', title="Income Distribution per Zipcode in the City of Buffalo")
    return html.Ul([html.Li(f"{key}: {value}") for key, value in city_dict.items()]), fig

@app.callback(
    [Output('suburb-output', 'children'),
    Output('barchart-suburb', 'figure')],
    Input('suburb-button', 'n_clicks'),
)
def update_suburb_stats(n_clicks):
    sub_dict = {"Average_Stores_Per Zipcode": "No Data Available"}
    fig = go.Figure() 
    if n_clicks > 0:
        sub_dict = {'Average_Stores_Per Zipcode': sub_avg['Average_Stores_Per_Zipcode'].iloc[0]}
        fig = px.bar(sub_income_query, x=['Returns_Under_25k', 'Returns_25k_to_50k', 'Returns_50k_to_75k', 'Returns_75k_to_100k', 'Returns_100k_to_200k', 'Returns_Above_200k'],
                        y='Zipcode', title="Income Distribution per Zipcode in the Suburbs")
    return html.Ul([html.Li(f"{key}: {value}") for key, value in sub_dict.items()]), fig

@app.callback(
    Output('last-graph', 'figure'),
    Input('last-buttons', 'value'))

def update_graph(region):
    fig = go.Figure()
    if region == 'City of Buffalo':
        fig = px.line(focus_city, x="Store_Focus", y="Number_of_Stores", title='Quality of Food Stores in the City of Buffalo')
    else:
        fig = px.line(focus_sub[1:7], x="Store_Focus", y="Number_of_Stores", title='Quality of Food Stores in the Suburbs')
    return fig
    
if __name__ == '__main__':
    app.run_server(debug=True)